In [ ]:
import pandas as pd
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import scipy.spatial
from sklearn.metrics.pairwise import haversine_distances
from math import radians
import osmnx as ox
import networkx as nx
import scipy.sparse
from collections import defaultdict
import igraph as ig
import numpy as np
import pytz
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
data_path = "Fire_Department_Calls_for_Service.csv"
station_path  = "fire_base_stations.csv"
hospital_path  = "hospitals_sfc.csv"

date_columns = ["Dispatch DtTm", "Response DtTm", "On Scene DtTm", "Transport DtTm", "Hospital DtTm", "Available DtTm", "Received DtTm"]
dtm_format = "%m/%d/%Y %I:%M:%S %p"
time_zone = pytz.timezone('America/Los_Angeles') #sf time zone is LA
data = pd.read_csv(data_path, parse_dates=False, low_memory=False)


for column in date_columns:
    data[column] = pd.to_datetime(data[column], format=dtm_format, utc=True) #.dt.tz_localize(time_zone)

data.head(100)

In [ ]:
#load stations
stations = pd.read_csv(station_path)
stations

#Hospitals
Remove Seton and Kaiser SSF because these hospitals are outside of SF and the area where all the incidents happen

In [ ]:
#load hospitals
hospitals_to_exclude = ["Seton", "Kaiser SSF"]
hospitals = pd.read_csv(hospital_path)
hospitals = hospitals[~hospitals["HospitalID"].isin(hospitals_to_exclude)]
hospitals = hospitals.reset_index()
hospitals

In [ ]:
#bounding_box
north_lat = 37.831666230626
south_lat = 37.70825596126
east_lon = -122.36147482652916
west_lon = -122.513648358854
#[37.70825596126, -122.513648358854] [37.831666230626, -122.36147482652916]

Things to filter:
-Response DtTM not na
-Unit Type medic

Things which we ignore: private ambulances

In [ ]:
data = data[~data["Response DtTm"].isnull()]
data = data[data["Unit Type"] == "MEDIC"]
data = data[~data["Zipcode of Incident"].isnull()]


In [ ]:
data = data[(data["Response DtTm"].dt.year >= 2001) & (data["Response DtTm"].dt.year <= 2021)]


Filter out incidents that do not follow a logical time order

In [ ]:
data = data[~(data["Response DtTm"]>data["On Scene DtTm"])]
data = data[~(data["On Scene DtTm"] > data["Transport DtTm"])]
data = data[~(data["Transport DtTm"] > data["Hospital DtTm"])]
data = data[~(data["Hospital DtTm"] > data["Available DtTm"])]
data = data[~(data["Response DtTm"] > data["Available DtTm"])]

In [ ]:
data["lon"] = data["case_location"].str.replace("POINT (", "", regex=False).str.replace(")", "", regex=False).str.split(" ").str.get(0).astype(float)
data["lat"] = data["case_location"].str.replace("POINT (", "", regex=False).str.replace(")", "", regex=False).str.split(" ").str.get(1).astype(float)

In [ ]:
data = data[~(data["lat"].isnull() | data["lon"].isnull())]

In [ ]:
data = data.copy()

In [ ]:
data["seconds_to_incident"] = (data["On Scene DtTm"] - data["Response DtTm"]).dt.total_seconds()

In [ ]:
data["seconds_at_incident"] = (data["Transport DtTm"] - data["On Scene DtTm"]).dt.total_seconds()
data["seconds_to_hospital"] = (data["Hospital DtTm"] - data["Transport DtTm"]).dt.total_seconds()
data["seconds_hospital_to_available"] = (data["Available DtTm"] - data["Hospital DtTm"]).dt.total_seconds()
data["patient_went_to_hospital"] = ~data["Transport DtTm"].isna()

In [ ]:
data["Neighborhooods - Analysis Boundaries"].unique()

In [ ]:
data["Call Type"].unique()

In [ ]:
data[data["Call Type"] == "Aircraft Emergency"]

In [ ]:
data[data["Neighborhooods - Analysis Boundaries"] == "None"]

In [ ]:
data["total_emergency_time_seconds"] = (data["Available DtTm"] - data["Response DtTm"]).dt.total_seconds()


In [ ]:
data["month"] = data["Response DtTm"].dt.month
incidents_per_month = data.groupby("month").size().reset_index(name="count")
incidents_per_month


In [ ]:
sns.barplot(data=incidents_per_month, x="month", y="count")


In [ ]:
data["minute_of_day"] = (
            (data["Response DtTm"] - data["Response DtTm"].dt.normalize()) / pd.Timedelta("15 minute")).astype(int)
incidents_per_minute = data.groupby(["minute_of_day",
                                     data["Response DtTm"].dt.day_of_year]).size().reset_index(name="count")
sns.lineplot(data=incidents_per_minute, x="minute_of_day", y="count")


In [ ]:
data["day_of_week"] = data["Response DtTm"].dt.day_of_week
incidents_per_minute = data.groupby(["day_of_week",
                                     data["Response DtTm"].dt.isocalendar().week]).size().reset_index(name="count")
sns.barplot(data=incidents_per_minute, x="day_of_week", y="count")



# Find out bounding box of the data

In [ ]:
min_long = data["lon"].min()
max_long = data["lon"].max()
min_lat = data["lat"].min()
max_lat = data["lat"].max()

print([min_lat, min_long],[max_lat,max_long])

m = folium.Map()
m.fit_bounds([[min_lat, min_long],[max_lat,max_long]])
#folium.LayerControl().add_to(m)
folium.TileLayer('openstreetmap').add_to(m)
m
3

In [ ]:
incidents_fg = folium.map.FeatureGroup(name="Incidents")
plugins.FastMarkerCluster(data[["lat", "lon"]]).add_to(incidents_fg)
m.add_child(incidents_fg)

hospitals_fg = folium.map.FeatureGroup(name="Hospitals")
for _, row in hospitals.iterrows():
    hospitals_fg.add_child(folium.CircleMarker(location=[row["latitude"], row["longitude"]], color="red"))
m.add_child(hospitals_fg)

stations_fg = folium.map.FeatureGroup(name="Stations")
for _, row in stations.iterrows():
    stations_fg.add_child(folium.CircleMarker(location=[row["lat"], row["lon"]]))
m.add_child(stations_fg)
3

In [ ]:
incidents_per_point = data.groupby(["lat", "lon"]).size().sort_values(ascending=False) .reset_index(name='count')


In [ ]:
heatmap_fg = folium.map.FeatureGroup(name="heatmap")
plugins.HeatMap(incidents_per_point).add_to(heatmap_fg)
m.add_child(heatmap_fg)
3

In [ ]:
m.add_child(folium.LayerControl())

In [ ]:
use_stored_graph = True

if not use_stored_graph:
    #network_type = "drive_service"
    network_type = "drive"
    graph = ox.graph_from_bbox(north=north_lat,
                               south=south_lat,
                               east=east_lon,
                               west=west_lon,
                               network_type=network_type)
    graph = ox.speed.add_edge_speeds(graph)
    graph = ox.speed.add_edge_travel_times(graph)
    #ox.basic_stats(graph)

if not use_stored_graph:
    edges_before_strong = len(graph.edges())
    nodes_before_strong = len(graph.nodes())
    largest_comp = max(nx.strongly_connected_components(graph), key=len)
    graph = graph.subgraph(largest_comp).copy()

    edges_after_strong = len(graph.edges())
    nodes_after_strong = len(graph.nodes())
    print(edges_before_strong - edges_after_strong)
    print(nodes_before_strong - nodes_after_strong)

    if not use_stored_graph:
        osmids = list(graph.nodes)
        graph = nx.relabel.convert_node_labels_to_integers(graph)

        # give each node its original osmid as attribute since we relabeled them
        osmid_values = {k: v for k, v in zip(graph.nodes, osmids)}
        nx.set_node_attributes(graph, osmid_values, "osmid")


if not use_stored_graph:
    ox.save_graphml(graph, "sf/graph.gz")
else:
    graph = ox.load_graphml("sf/graph.gz")
    osmids = list(graph.nodes)

# convert networkx graph to igraph
G_ig = ig.Graph(directed=True)
G_ig.add_vertices(graph.nodes)
G_ig.add_edges(graph.edges())
G_ig.vs["osmid"] = osmids
G_ig.es["travel_time"] = list(nx.get_edge_attributes(graph, "travel_time").values())

In [ ]:
nearest_nodes_stations = ox.nearest_nodes(graph, stations["lon"], stations["lat"])
nearest_nodes_incidents = ox.nearest_nodes(graph, data["lon"], data["lat"])
nearest_nodes_hospitals = ox.nearest_nodes(graph, hospitals["longitude"], hospitals["latitude"])

stations["nearest_node"] = nearest_nodes_stations
data["nearest_node"] = nearest_nodes_incidents
hospitals["nearest_node"] = nearest_nodes_hospitals

#data["position_in_matrix"] = data.index
stations["position_in_matrix"] = stations.index
hospitals["position_in_matrix"] = hospitals.index

assert len(hospitals[hospitals.duplicated(subset=["nearest_node"], keep=False)]) == 0
assert len(stations[stations.duplicated(subset=["nearest_node"], keep=False)]) == 0

In [ ]:
unique_incident_nodes = np.array(nearest_nodes_incidents)
unique_incident_nodes = np.unique(unique_incident_nodes)
unique_incident_nodes_df = pd.DataFrame()
unique_incident_nodes_df["nearest_node_unique"] = unique_incident_nodes
unique_incident_nodes_df["position_in_matrix"] = unique_incident_nodes_df.index

#data = data.merge(unique_incident_nodes_df, left_on="nearest_node", right_on="nearest_node_unique")

In [ ]:
station_to_incident = np.array(G_ig.shortest_paths(source=nearest_nodes_stations, target=unique_incident_nodes, weights="travel_time"))
incident_to_hospitals = np.array(G_ig.shortest_paths(source=unique_incident_nodes, target=nearest_nodes_hospitals, weights="travel_time"))
incident_to_station = np.array(G_ig.shortest_paths(source=unique_incident_nodes,
                                                  target=nearest_nodes_stations,
                                                  weights="travel_time"))


In [ ]:
hospital_to_station = np.array(G_ig.shortest_paths(source=nearest_nodes_hospitals,
                                                   target=nearest_nodes_stations,
                                                   weights="travel_time"))
station_to_station = np.array(G_ig.shortest_paths(source=nearest_nodes_stations,
                                                  target=nearest_nodes_stations,
                                                  weights="travel_time"))


In [ ]:
#find the closest station for each incident

#format station_to_incident : index of node to index of node
closest_station = station_to_incident.argmin(axis=0)
unique_incident_nodes_df["closest_station_index"] = closest_station
data = data.merge(unique_incident_nodes_df, left_on="nearest_node", right_on="nearest_node_unique")

# Split data
Train = 2001-2019
Validation = 2020
Test = 2021

In [ ]:
validation_data = data[data["Response DtTm"].dt.year == 2020]
test_data = data[data["Response DtTm"].dt.year == 2021]
train_data = data[(data["Response DtTm"].dt.year >= 2001) & (data["Response DtTm"].dt.year <= 2019)]

In [ ]:
data.groupby(data["Response DtTm"].dt.year).size()

# Export Data

In [ ]:
data_path = "processed"

#export travel times
np.savez(f"{data_path}/station_to_incident",station_to_incident)
np.savez(f"{data_path}/incident_to_station",incident_to_station)
np.savez(f"{data_path}/incident_to_hospitals",incident_to_hospitals)
np.savez(f"{data_path}/hospital_to_station",hospital_to_station)
np.savez(f"{data_path}/station_to_station", station_to_station)


#time format int, seconds since midnight
#position in distance matrix
#day of year

def create_export_frame(base_df):
    data_export = pd.DataFrame()
    data_export["IncidentTime"] = ((
                (base_df["Response DtTm"] - base_df["Response DtTm"].dt.normalize()) / pd.Timedelta("1s")).astype(int)) + (base_df["Response DtTm"].dt.day_of_year * 24*60*60)
    data_export["DayOfYear"] = base_df["Response DtTm"].dt.day_of_year
    data_export["PositionInDistanceMatrix"] = base_df["position_in_matrix_x"]
    data_export["DemandLocationID"] = base_df["closest_station_index"]
    data_export["Month"] = base_df["Response DtTm"].dt.month
    data_export["WeekDay"] = base_df["Response DtTm"].dt.day_of_week
    data_export["Year"] = base_df["Response DtTm"].dt.year
    data_export["EpisodeGroup"] = base_df["Response DtTm"].dt.year
    data_export["Lon"] = base_df["lon"]
    data_export["Lat"] = base_df["lat"]
    data_export["Priority"] = base_df["Final Priority"]
    data_export["TransportToHospital"] = base_df["patient_went_to_hospital"]
    data_export["SecondsAtIncident"] = base_df["seconds_at_incident"]
    data_export["TotalEmergencyTime"] = base_df["total_emergency_time_seconds"]
    data_export["TimeAtHospital"] = base_df["seconds_hospital_to_available"]
    return data_export


def create_episode_groups(base_df):
    years = base_df["Response DtTm"].dt.year.unique()

    data_export = pd.DataFrame()
    data_export["EpisodeGroup"] = years

    data_export["MaxTime"] = [(base_df[base_df["Response DtTm"].dt.year == year]["Response DtTm"].dt.day_of_year.max() + 1) * 24*60*60 for year in years]

    return data_export


train_export = create_export_frame(train_data)
val_export = create_export_frame(validation_data)
test_export = create_export_frame(test_data)

train_export.to_csv(f"{data_path}/train.csv")
val_export.to_csv(f"{data_path}/val.csv")
test_export.to_csv(f"{data_path}/test.csv")

create_episode_groups(train_data).to_csv(f"{data_path}/episode_groups_train.csv")
create_episode_groups(validation_data).to_csv(f"{data_path}/episode_groups_val.csv")
create_episode_groups(test_data).to_csv(f"{data_path}/episode_groups_test.csv")


hospital_export = pd.DataFrame()
hospital_export["Name"] = hospitals["HospitalID"]
hospital_export["PositionInDistanceMatrix"] = hospitals["position_in_matrix"]
hospital_export["Lon"] = hospitals["longitude"]
hospital_export["Lat"] = hospitals["latitude"]
hospital_export.to_csv(f"{data_path}/hospitals.csv")


station_export = pd.DataFrame()
station_export["Name"] = stations["name"]
station_export["PositionInDistanceMatrix"] = stations["position_in_matrix"]
station_export["Lon"] = stations["lon"]
station_export["Lat"] = stations["lat"]
station_export.to_csv(f"{data_path}/stations.csv")

In [ ]:
count_per_station = validation_data.groupby("closest_station_index").size().reset_index(name="count")
total = len(validation_data)
count_per_station["fraction"] = count_per_station["count"]/total

count_per_station.to_csv(f"{data_path}/demand_weights.csv")

count_per_station

In [ ]:
# number of incidents per hour
# we should only do this on the validation set
validation_data["time"] = pd.DatetimeIndex(validation_data["Response DtTm"])
validation_data = validation_data.set_index("time")

incidents_per_hour_and_demand_location = validation_data.groupby("closest_station_index").resample("60min").size().reset_index(
    name="count")

average_number_of_incidents_per_hour_per_station = incidents_per_hour_and_demand_location.groupby(
        "closest_station_index").mean()

average_number_of_incidents_per_hour_per_station


In [ ]:
demand_forcast = incidents_per_hour_and_demand_location.groupby(["closest_station_index",
                                                                 incidents_per_hour_and_demand_location.time.dt.hour]).mean()
demand_forcast = demand_forcast.to_numpy().reshape(len(stations), 24)
np.savez(f"{data_path}/average_hourly_demand", demand_forcast)


In [ ]:
data[(data["seconds_hospital_to_available"]>0) & ~data["Hospital DtTm"].isna()]["seconds_hospital_to_available"].describe()